In [1]:
import datetime
import utils
import pymysql
import sshtunnel
import pandas as pd 
import numpy as np 
from pythermalcomfort.models import pmv_ppd
from pythermalcomfort.utilities import v_relative, clo_dynamic
from math import exp, sqrt
import os
from datetime import datetime
import json

In [2]:
import datetime
# tabla
tabla = "data"

# Variables seleccionadas tabla
#ToDo: Revisar por qué se convierte a minusc. Tiene pinta de alguna configuración de la bbdd
variables = ["UPPER(id_sensor) AS id_sensor","variable","value","unit","timestamp"] #upper porque se cogen convierten los datos a minusc.

# Id sensores
id_sensores = ["70B3D57BA0001C2D","QUCONTFUAU1AULA1001"]
sql_filtro_sensores = " OR ".join([ f"id_sensor = '{id}'" for id in id_sensores])

# Definir la fecha de inicio a partir de la cual seleccionamos los datos
now = datetime.datetime.now()
fecha_inicio = now - datetime.timedelta(minutes=20)
fecha_inicio_bd = fecha_inicio - datetime.timedelta(hours=2)  # Ajuste de la diferencia horaria


# Query
query = f"""
SELECT 
    {",".join(variables)}
FROM 
    {tabla}
WHERE 
    ({sql_filtro_sensores})
    AND timestamp >= '{fecha_inicio_bd.strftime('%Y-%m-%d %H:%M:%S')}'
ORDER BY TIMESTAMP DESC
"""

In [3]:
print(query)


SELECT 
    UPPER(id_sensor) AS id_sensor,variable,value,unit,timestamp
FROM 
    data
WHERE 
    (id_sensor = '70B3D57BA0001C2D' OR id_sensor = 'QUCONTFUAU1AULA1001')
    AND timestamp >= '2023-07-11 14:11:35'
ORDER BY TIMESTAMP DESC



### Credenciales

In [4]:
# Abro el archivo de sql.json en creds y lo cargo. Esto me devuelve un diccionario de python
with open('creds/sql.json', 'r') as json_file:
    creds_sql = json.load(json_file)

## Importación

In [5]:
# get_data_ssh es una función que crea directamente el DataFrame de python al poner la query 
# en sql. Es una función propio del script utils en el proyecto 
df_imported = utils.get_data_ssh(query, database="smartcampus_demo", **creds_sql)
df_imported

,id_sensor,variable,value,unit,timestamp
0,70B3D57BA0001C2D,Total VOC,84.0000,ppb,2023-07-11 14:28:19
1,70B3D57BA0001C2D,Activity counter,0.0000,None,2023-07-11 14:28:19
2,70B3D57BA0001C2D,Raw IR reading,37902.0000,None,2023-07-11 14:28:19
3,70B3D57BA0001C2D,CO2 sensor status,0.0000,None,2023-07-11 14:28:19
4,70B3D57BA0001C2D,CO2 concentration,413.0000,ppm,2023-07-11 14:28:19
5,70B3D57BA0001C2D,Illuminance,0.0000,lx,2023-07-11 14:28:19
6,70B3D57BA0001C2D,Ambient light (infrared),0.0000,None,2023-07-11 14:28:19
7,70B3D57BA0001C2D,Ambient light (visible + infrared),0.0000,None,2023-07-11 14:28:19
8,70B3D57BA0001C2D,Barometric pressure,93822.0000,Pa,2023-07-11 14:28:19
9,70B3D57BA0001C2D,Air humidity,35.5184,%,2023-07-11 14:28:19


In [6]:
df = df_imported.copy()
df

,id_sensor,variable,value,unit,timestamp
0,70B3D57BA0001C2D,Total VOC,84.0000,ppb,2023-07-11 14:28:19
1,70B3D57BA0001C2D,Activity counter,0.0000,None,2023-07-11 14:28:19
2,70B3D57BA0001C2D,Raw IR reading,37902.0000,None,2023-07-11 14:28:19
3,70B3D57BA0001C2D,CO2 sensor status,0.0000,None,2023-07-11 14:28:19
4,70B3D57BA0001C2D,CO2 concentration,413.0000,ppm,2023-07-11 14:28:19
5,70B3D57BA0001C2D,Illuminance,0.0000,lx,2023-07-11 14:28:19
6,70B3D57BA0001C2D,Ambient light (infrared),0.0000,None,2023-07-11 14:28:19
7,70B3D57BA0001C2D,Ambient light (visible + infrared),0.0000,None,2023-07-11 14:28:19
8,70B3D57BA0001C2D,Barometric pressure,93822.0000,Pa,2023-07-11 14:28:19
9,70B3D57BA0001C2D,Air humidity,35.5184,%,2023-07-11 14:28:19


# Limpieza de Datos

In [7]:
df[df.duplicated()]

,id_sensor,variable,value,unit,timestamp


In [8]:
# Quitamos esos duplicados 
df = df.drop_duplicates()
df

,id_sensor,variable,value,unit,timestamp
0,70B3D57BA0001C2D,Total VOC,84.0000,ppb,2023-07-11 14:28:19
1,70B3D57BA0001C2D,Activity counter,0.0000,None,2023-07-11 14:28:19
2,70B3D57BA0001C2D,Raw IR reading,37902.0000,None,2023-07-11 14:28:19
3,70B3D57BA0001C2D,CO2 sensor status,0.0000,None,2023-07-11 14:28:19
4,70B3D57BA0001C2D,CO2 concentration,413.0000,ppm,2023-07-11 14:28:19
5,70B3D57BA0001C2D,Illuminance,0.0000,lx,2023-07-11 14:28:19
6,70B3D57BA0001C2D,Ambient light (infrared),0.0000,None,2023-07-11 14:28:19
7,70B3D57BA0001C2D,Ambient light (visible + infrared),0.0000,None,2023-07-11 14:28:19
8,70B3D57BA0001C2D,Barometric pressure,93822.0000,Pa,2023-07-11 14:28:19
9,70B3D57BA0001C2D,Air humidity,35.5184,%,2023-07-11 14:28:19


In [9]:
#Una vez esté todo normalizado a la misma unidad, nos podemos cargar la columna, pues es innecesaria:
df.drop(columns="unit",inplace=True)
df

,id_sensor,variable,value,timestamp
0,70B3D57BA0001C2D,Total VOC,84.0000,2023-07-11 14:28:19
1,70B3D57BA0001C2D,Activity counter,0.0000,2023-07-11 14:28:19
2,70B3D57BA0001C2D,Raw IR reading,37902.0000,2023-07-11 14:28:19
3,70B3D57BA0001C2D,CO2 sensor status,0.0000,2023-07-11 14:28:19
4,70B3D57BA0001C2D,CO2 concentration,413.0000,2023-07-11 14:28:19
5,70B3D57BA0001C2D,Illuminance,0.0000,2023-07-11 14:28:19
6,70B3D57BA0001C2D,Ambient light (infrared),0.0000,2023-07-11 14:28:19
7,70B3D57BA0001C2D,Ambient light (visible + infrared),0.0000,2023-07-11 14:28:19
8,70B3D57BA0001C2D,Barometric pressure,93822.0000,2023-07-11 14:28:19
9,70B3D57BA0001C2D,Air humidity,35.5184,2023-07-11 14:28:19


## Conversión de la fecha a timestamp

In [10]:
df["timestamp"] = pd.to_datetime(df["timestamp"])

In [11]:
df.dtypes

id_sensor            object
variable             object
value               float64
timestamp    datetime64[ns]
dtype: object

## Normalización y redondeo del tiempo

In [12]:
from datetime import datetime, timedelta

def round_minutes(date, interval=5):
    # date: se espera que tenga la forma '%Y-%m-%d %H:%M'
    dt = datetime.strptime(date, '%Y-%m-%d %H:%M')
    
    rounded_minute = (dt.minute // interval) * interval
    if dt.minute % interval >= interval / 2:
        rounded_minute += interval
        
    if rounded_minute == 60:
        rounded_dt = dt.replace(minute=0) + timedelta(hours=1)
    else:
        rounded_dt = dt.replace(minute=rounded_minute)
        
    rounded_dt = rounded_dt.replace(second=0, microsecond=0)
    
    return rounded_dt.strftime('%Y-%m-%d %H:%M')

In [13]:
# Aplicamos la función anterior y guardamos la columna anterior para hacer checkeos si fuera 
# necesario en algún momento 
df["timestamp_original"] = df["timestamp"] 
# df["timestamp"] = pd.to_datetime(df["timestamp"])
df["timestamp"] = [round_minutes(date) for date in df["timestamp"].dt.strftime('%Y-%m-%d %H:%M')]
df["timestamp"] = pd.to_datetime(df["timestamp"])
df

,id_sensor,variable,value,timestamp,timestamp_original
0,70B3D57BA0001C2D,Total VOC,84.0000,2023-07-11 14:30:00,2023-07-11 14:28:19
1,70B3D57BA0001C2D,Activity counter,0.0000,2023-07-11 14:30:00,2023-07-11 14:28:19
2,70B3D57BA0001C2D,Raw IR reading,37902.0000,2023-07-11 14:30:00,2023-07-11 14:28:19
3,70B3D57BA0001C2D,CO2 sensor status,0.0000,2023-07-11 14:30:00,2023-07-11 14:28:19
4,70B3D57BA0001C2D,CO2 concentration,413.0000,2023-07-11 14:30:00,2023-07-11 14:28:19
5,70B3D57BA0001C2D,Illuminance,0.0000,2023-07-11 14:30:00,2023-07-11 14:28:19
6,70B3D57BA0001C2D,Ambient light (infrared),0.0000,2023-07-11 14:30:00,2023-07-11 14:28:19
7,70B3D57BA0001C2D,Ambient light (visible + infrared),0.0000,2023-07-11 14:30:00,2023-07-11 14:28:19
8,70B3D57BA0001C2D,Barometric pressure,93822.0000,2023-07-11 14:30:00,2023-07-11 14:28:19
9,70B3D57BA0001C2D,Air humidity,35.5184,2023-07-11 14:30:00,2023-07-11 14:28:19


In [14]:
df= df.groupby(["id_sensor","timestamp","variable"])["value"].mean().reset_index()
df

,id_sensor,timestamp,variable,value
0,70B3D57BA0001C2D,2023-07-11 14:20:00,Activity counter,0.0000
1,70B3D57BA0001C2D,2023-07-11 14:20:00,Air humidity,35.5917
2,70B3D57BA0001C2D,2023-07-11 14:20:00,Air temperature,27.4968
3,70B3D57BA0001C2D,2023-07-11 14:20:00,Ambient light (infrared),0.0000
4,70B3D57BA0001C2D,2023-07-11 14:20:00,Ambient light (visible + infrared),0.0000
5,70B3D57BA0001C2D,2023-07-11 14:20:00,Barometric pressure,93826.0000
6,70B3D57BA0001C2D,2023-07-11 14:20:00,Battery voltage,2.7340
7,70B3D57BA0001C2D,2023-07-11 14:20:00,CO2 concentration,409.0000
8,70B3D57BA0001C2D,2023-07-11 14:20:00,CO2 sensor status,0.0000
9,70B3D57BA0001C2D,2023-07-11 14:20:00,Illuminance,0.0000


In [15]:
df = pd.pivot_table(df,index=["timestamp","id_sensor"],columns=["variable"]).reset_index()
df

timestamp            id_sensor            value  \
variable                                          Activity counter   
0        2023-07-11 14:15:00  QUCONTFUAU1AULA1001              NaN   
1        2023-07-11 14:20:00     70B3D57BA0001C2D              0.0   
2        2023-07-11 14:20:00  QUCONTFUAU1AULA1001              NaN   
3        2023-07-11 14:30:00     70B3D57BA0001C2D              0.0   

                                                                \
variable Air humidity Air temperature Ambient light (infrared)   
0                 NaN             NaN                      NaN   
1             35.5917         27.4968                      0.0   
2                 NaN             NaN                      NaN   
3             35.5184         27.5021                      0.0   

                                                                 \
variable Ambient light (visible + infrared) Barometric pressure   
0                                       NaN                 NaN   
1                                       0.0             93826.0   
2                                       NaN                 NaN   
3                                       0.0             93822.0   

                                                                          \
variable Battery voltage CO2 concentration CO2 sensor status Illuminance   
0                    NaN               NaN               NaN         NaN   
1                  2.734             409.0               0.0         0.0   
2                    NaN               NaN               NaN         NaN   
3                  2.734             413.0               0.0         0.0   

                                             
variable Occupancy Raw IR reading Total VOC  
0              0.0            NaN       NaN  
1              NaN        37918.0      86.0  
2              0.0            NaN       NaN  
3              NaN        37902.0      84.0

In [16]:
df.columns = [ col[1] if col[1]!='' else col[0] for col in df.columns]
df

,timestamp,id_sensor,Activity counter,Air humidity,Air temperature,Ambient light (infrared),Ambient light (visible + infrared),Barometric pressure,Battery voltage,CO2 concentration,CO2 sensor status,Illuminance,Occupancy,Raw IR reading,Total VOC
0,2023-07-11 14:15:00,QUCONTFUAU1AULA1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,2023-07-11 14:20:00,70B3D57BA0001C2D,0.0,35.5917,27.4968,0.0,0.0,93826.0,2.734,409.0,0.0,0.0,NaN,37918.0,86.0
2,2023-07-11 14:20:00,QUCONTFUAU1AULA1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,2023-07-11 14:30:00,70B3D57BA0001C2D,0.0,35.5184,27.5021,0.0,0.0,93822.0,2.734,413.0,0.0,0.0,NaN,37902.0,84.0


## Unificación de la información de sensores distintos en un mismo instante de tiempo

In [17]:
df_join_sensors = df.drop(columns="id_sensor").groupby("timestamp").sum(min_count=1).reset_index()
df_join_sensors["id_sensor"] = df.groupby("timestamp")["id_sensor"].apply(lambda ids: "&".join(ids)).reset_index(drop=True)
df = df_join_sensors.copy()
df

,timestamp,Activity counter,Air humidity,Air temperature,Ambient light (infrared),Ambient light (visible + infrared),Barometric pressure,Battery voltage,CO2 concentration,CO2 sensor status,Illuminance,Occupancy,Raw IR reading,Total VOC,id_sensor
0,2023-07-11 14:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,QUCONTFUAU1AULA1001
1,2023-07-11 14:20:00,0.0,35.5917,27.4968,0.0,0.0,93826.0,2.734,409.0,0.0,0.0,0.0,37918.0,86.0,70B3D57BA0001C2D&QUCONTFUAU1AULA1001
2,2023-07-11 14:30:00,0.0,35.5184,27.5021,0.0,0.0,93822.0,2.734,413.0,0.0,0.0,NaN,37902.0,84.0,70B3D57BA0001C2D


In [18]:
df = df.sort_values('timestamp')

# Rellenar los valores NaN con los valores de la fila anterior (forward fill)
df = df.fillna(method='ffill')
df

,timestamp,Activity counter,Air humidity,Air temperature,Ambient light (infrared),Ambient light (visible + infrared),Barometric pressure,Battery voltage,CO2 concentration,CO2 sensor status,Illuminance,Occupancy,Raw IR reading,Total VOC,id_sensor
0,2023-07-11 14:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,QUCONTFUAU1AULA1001
1,2023-07-11 14:20:00,0.0,35.5917,27.4968,0.0,0.0,93826.0,2.734,409.0,0.0,0.0,0.0,37918.0,86.0,70B3D57BA0001C2D&QUCONTFUAU1AULA1001
2,2023-07-11 14:30:00,0.0,35.5184,27.5021,0.0,0.0,93822.0,2.734,413.0,0.0,0.0,0.0,37902.0,84.0,70B3D57BA0001C2D


## Imputación de valores que llegan irregularmente

In [19]:
dates_imputed = pd.DataFrame(
    {'timestamp_imputed' : pd.date_range(start = df.iloc[0]["timestamp"], 
                                         end = df.iloc[-1]["timestamp"],
                                         freq= "5min")}
)
dates_imputed

,timestamp_imputed
0,2023-07-11 14:15:00
1,2023-07-11 14:20:00
2,2023-07-11 14:25:00
3,2023-07-11 14:30:00


In [20]:
df_fill_imputed = dates_imputed.merge(df,
                                      how="left", 
                                      left_on = "timestamp_imputed", 
                                      right_on ="timestamp")
df_fill_imputed

,timestamp_imputed,timestamp,Activity counter,Air humidity,Air temperature,Ambient light (infrared),Ambient light (visible + infrared),Barometric pressure,Battery voltage,CO2 concentration,CO2 sensor status,Illuminance,Occupancy,Raw IR reading,Total VOC,id_sensor
0,2023-07-11 14:15:00,2023-07-11 14:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,QUCONTFUAU1AULA1001
1,2023-07-11 14:20:00,2023-07-11 14:20:00,0.0,35.5917,27.4968,0.0,0.0,93826.0,2.734,409.0,0.0,0.0,0.0,37918.0,86.0,70B3D57BA0001C2D&QUCONTFUAU1AULA1001
2,2023-07-11 14:25:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-07-11 14:30:00,2023-07-11 14:30:00,0.0,35.5184,27.5021,0.0,0.0,93822.0,2.734,413.0,0.0,0.0,0.0,37902.0,84.0,70B3D57BA0001C2D


In [21]:
columns_to_impute = ["Air humidity", "Air temperature", "CO2 concentration", "Occupancy"]
for col in columns_to_impute:
    df_fill_imputed[f"{col}_imputed"] = df_fill_imputed[f"{col}"].interpolate(method="linear")
df_fill_imputed

,timestamp_imputed,timestamp,Activity counter,Air humidity,Air temperature,Ambient light (infrared),Ambient light (visible + infrared),Barometric pressure,Battery voltage,CO2 concentration,CO2 sensor status,Illuminance,Occupancy,Raw IR reading,Total VOC,id_sensor,Air humidity_imputed,Air temperature_imputed,CO2 concentration_imputed,Occupancy_imputed
0,2023-07-11 14:15:00,2023-07-11 14:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,QUCONTFUAU1AULA1001,NaN,NaN,NaN,0.0
1,2023-07-11 14:20:00,2023-07-11 14:20:00,0.0,35.5917,27.4968,0.0,0.0,93826.0,2.734,409.0,0.0,0.0,0.0,37918.0,86.0,70B3D57BA0001C2D&QUCONTFUAU1AULA1001,35.59170,27.49680,409.0,0.0
2,2023-07-11 14:25:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.55505,27.49945,411.0,0.0
3,2023-07-11 14:30:00,2023-07-11 14:30:00,0.0,35.5184,27.5021,0.0,0.0,93822.0,2.734,413.0,0.0,0.0,0.0,37902.0,84.0,70B3D57BA0001C2D,35.51840,27.50210,413.0,0.0


In [22]:
# Falta imputar el id del sensor:
df_fill_imputed["id_sensor"] = np.where(df_fill_imputed["id_sensor"].isna(),"A81758FFFE057FDD",df_fill_imputed["id_sensor"])

In [23]:
# actializo la variable 
df = df_fill_imputed.copy()
df

,timestamp_imputed,timestamp,Activity counter,Air humidity,Air temperature,Ambient light (infrared),Ambient light (visible + infrared),Barometric pressure,Battery voltage,CO2 concentration,CO2 sensor status,Illuminance,Occupancy,Raw IR reading,Total VOC,id_sensor,Air humidity_imputed,Air temperature_imputed,CO2 concentration_imputed,Occupancy_imputed
0,2023-07-11 14:15:00,2023-07-11 14:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,QUCONTFUAU1AULA1001,NaN,NaN,NaN,0.0
1,2023-07-11 14:20:00,2023-07-11 14:20:00,0.0,35.5917,27.4968,0.0,0.0,93826.0,2.734,409.0,0.0,0.0,0.0,37918.0,86.0,70B3D57BA0001C2D&QUCONTFUAU1AULA1001,35.59170,27.49680,409.0,0.0
2,2023-07-11 14:25:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A81758FFFE057FDD,35.55505,27.49945,411.0,0.0
3,2023-07-11 14:30:00,2023-07-11 14:30:00,0.0,35.5184,27.5021,0.0,0.0,93822.0,2.734,413.0,0.0,0.0,0.0,37902.0,84.0,70B3D57BA0001C2D,35.51840,27.50210,413.0,0.0


## Últimas limpiezas y categorizaciones finales 

In [24]:
df = df\
    .rename(columns = {"Air temperature_imputed": "Temperatura",
                       "Air humidity_imputed": "Humedad",
                       "CO2 concentration_imputed": "CO2",
                       "timestamp_imputed":"Timestamp",
                       "Occupancy_imputed":"Ocupacion"
                      })[["id_sensor","Timestamp","Temperatura","Humedad","CO2","Ocupacion"]]
df

,id_sensor,Timestamp,Temperatura,Humedad,CO2,Ocupacion
0,QUCONTFUAU1AULA1001,2023-07-11 14:15:00,NaN,NaN,NaN,0.0
1,70B3D57BA0001C2D&QUCONTFUAU1AULA1001,2023-07-11 14:20:00,27.49680,35.59170,409.0,0.0
2,A81758FFFE057FDD,2023-07-11 14:25:00,27.49945,35.55505,411.0,0.0
3,70B3D57BA0001C2D,2023-07-11 14:30:00,27.50210,35.51840,413.0,0.0


In [25]:
#Tasa metabólica, para nuestras condiciones es 1.2
M = 1.2
#Velocidad del aire, estamos suponiendo que es 0.01
v = 0.01

df["Icl"] = np.where(
    (df['Timestamp'] > pd.Timestamp(2023,3,21)) & (df['Timestamp'] < pd.Timestamp(2023,9,21)),
    0.5,
    1
)
df

,id_sensor,Timestamp,Temperatura,Humedad,CO2,Ocupacion,Icl
0,QUCONTFUAU1AULA1001,2023-07-11 14:15:00,NaN,NaN,NaN,0.0,0.5
1,70B3D57BA0001C2D&QUCONTFUAU1AULA1001,2023-07-11 14:20:00,27.49680,35.59170,409.0,0.0,0.5
2,A81758FFFE057FDD,2023-07-11 14:25:00,27.49945,35.55505,411.0,0.0,0.5
3,70B3D57BA0001C2D,2023-07-11 14:30:00,27.50210,35.51840,413.0,0.0,0.5


In [26]:
df['IDA'] = np.where(pd.isnull(df['CO2']), None,
                     np.where(df['CO2'] <= 750, 'IDA 1',
                              np.where((df['CO2'] > 750) & (df['CO2'] <= 900), 'IDA 2',
                                       np.where((df['CO2'] > 900) & (df['CO2'] <= 1200), 'IDA 3', 'IDA 4')
                                       )
                              )
                     )

In [27]:
#Cálculo de la velocidad relativa del aire
v_r = v_relative(v=v, met= M)
v_r

array(0.07)

In [28]:
# Calcular el ITRC
df['Temperatura Radiante ITRC'] = 0.5 * df['Temperatura'] + 0.5 * df['Temperatura'] * df['Humedad'] / 100 + 1.2 * (df['Temperatura'] - df['Temperatura'] * df['Humedad'] / 100) - 10.7
df

,id_sensor,Timestamp,Temperatura,Humedad,CO2,Ocupacion,Icl,IDA,Temperatura Radiante ITRC
0,QUCONTFUAU1AULA1001,2023-07-11 14:15:00,NaN,NaN,NaN,0.0,0.5,None,NaN
1,70B3D57BA0001C2D&QUCONTFUAU1AULA1001,2023-07-11 14:20:00,27.49680,35.59170,409.0,0.0,0.5,IDA 1,29.193955
2,A81758FFFE057FDD,2023-07-11 14:25:00,27.49945,35.55505,411.0,0.0,0.5,IDA 1,29.204855
3,70B3D57BA0001C2D,2023-07-11 14:30:00,27.50210,35.51840,413.0,0.0,0.5,IDA 1,29.215756


In [29]:
# Esta celda tarda mucho. Lo mejor que se puede hacer cuando el tiempo de ejecución es largo es una 
# LIST COMPREHENSION
def own_pmv_ppd(temperatura, temperatura_radiante, humedad, icl):
    return pmv_ppd(tdb=temperatura, 
                   tr=temperatura_radiante, 
                   vr=v_r, 
                   rh=humedad, 
                   met=M, 
                   clo= clo_dynamic(clo=icl, met=M)
                  )

df2 = pd.DataFrame([own_pmv_ppd(temperatura, temperatura_globo, humedad, icl) 
                    for temperatura,temperatura_globo,humedad,icl 
                    in df[["Temperatura","Temperatura Radiante ITRC","Humedad","Icl"]].values])
df2

,pmv,ppd
0,NaN,NaN
1,1.03,27.3
2,1.03,27.4
3,1.03,27.5


In [30]:
df = pd.concat([df,df2], axis=1)
df

,id_sensor,Timestamp,Temperatura,Humedad,CO2,Ocupacion,Icl,IDA,Temperatura Radiante ITRC,pmv,ppd
0,QUCONTFUAU1AULA1001,2023-07-11 14:15:00,NaN,NaN,NaN,0.0,0.5,None,NaN,NaN,NaN
1,70B3D57BA0001C2D&QUCONTFUAU1AULA1001,2023-07-11 14:20:00,27.49680,35.59170,409.0,0.0,0.5,IDA 1,29.193955,1.03,27.3
2,A81758FFFE057FDD,2023-07-11 14:25:00,27.49945,35.55505,411.0,0.0,0.5,IDA 1,29.204855,1.03,27.4
3,70B3D57BA0001C2D,2023-07-11 14:30:00,27.50210,35.51840,413.0,0.0,0.5,IDA 1,29.215756,1.03,27.5


In [31]:
bins = [-np.inf, 6, 10, 15, np.inf]   # Define los puntos de corte para cada categoría
labels = ['A', 'B', 'C', 'D']         # Define las etiquetas para cada categoría
df['Categoria'] = pd.cut(df['ppd'], bins=bins, labels=labels, include_lowest=True)
df

,id_sensor,Timestamp,Temperatura,Humedad,CO2,Ocupacion,Icl,IDA,Temperatura Radiante ITRC,pmv,ppd,Categoria
0,QUCONTFUAU1AULA1001,2023-07-11 14:15:00,NaN,NaN,NaN,0.0,0.5,None,NaN,NaN,NaN,NaN
1,70B3D57BA0001C2D&QUCONTFUAU1AULA1001,2023-07-11 14:20:00,27.49680,35.59170,409.0,0.0,0.5,IDA 1,29.193955,1.03,27.3,D
2,A81758FFFE057FDD,2023-07-11 14:25:00,27.49945,35.55505,411.0,0.0,0.5,IDA 1,29.204855,1.03,27.4,D
3,70B3D57BA0001C2D,2023-07-11 14:30:00,27.50210,35.51840,413.0,0.0,0.5,IDA 1,29.215756,1.03,27.5,D


In [32]:
estrellas_dict = {
    ('IDA 1', 'A'): 5,
    ('IDA 2', 'A'): 5,
    ('IDA 1', 'B'): 4,
    ('IDA 2', 'B'): 4,
    ('IDA 1', 'C'): 3,
    ('IDA 2', 'C'): 3,
    ('IDA 3', 'A'): 2,
    ('IDA 3', 'B'): 2,
    ('IDA 3', 'C'): 1,
    ('IDA 4', 'A'): 0,
    ('IDA 4', 'B'): 0,
    ('IDA 4', 'C'): 0,
    ('IDA 1', 'D'): 0,
    ('IDA 2', 'D'): 0,
    ('IDA 3', 'D'): 0,
    ('IDA 4', 'D'): 0
}
df['Estrellas'] = df[['IDA', 'Categoria']].apply(lambda x: estrellas_dict.get(tuple(x)), axis=1)
df

,id_sensor,Timestamp,Temperatura,Humedad,CO2,Ocupacion,Icl,IDA,Temperatura Radiante ITRC,pmv,ppd,Categoria,Estrellas
0,QUCONTFUAU1AULA1001,2023-07-11 14:15:00,NaN,NaN,NaN,0.0,0.5,None,NaN,NaN,NaN,NaN,NaN
1,70B3D57BA0001C2D&QUCONTFUAU1AULA1001,2023-07-11 14:20:00,27.49680,35.59170,409.0,0.0,0.5,IDA 1,29.193955,1.03,27.3,D,0.0
2,A81758FFFE057FDD,2023-07-11 14:25:00,27.49945,35.55505,411.0,0.0,0.5,IDA 1,29.204855,1.03,27.4,D,0.0
3,70B3D57BA0001C2D,2023-07-11 14:30:00,27.50210,35.51840,413.0,0.0,0.5,IDA 1,29.215756,1.03,27.5,D,0.0


# Exportación de Datos

In [33]:
from sqlalchemy import create_engine
from sqlalchemy import text
import pymysql

In [34]:
# Leer las credenciales
with open('creds/TFG_credentials.json', 'r') as f:
    creds = json.load(f)

# Conexión con la BBDD 
engine = create_engine(f'mysql+pymysql://{creds["user"]}:{creds["password"]}@{creds["server"]}:{creds["port"]}/{creds["database"]}')

In [35]:
# Obtener las horas únicas del DataFrame (supongamos que está en una columna llamada 'hora')
horas_unicas = df['Timestamp'].unique()

In [36]:
# Iterar sobre las horas únicas y realizar la consulta y actualización para cada hora
for hora_especifica in horas_unicas:
    # Consultar los datos existentes para la misma hora en la base de datos
    consulta = f"SELECT * FROM Data_sensores WHERE Timestamp = '{hora_especifica}'"
    datos_existentes = pd.read_sql_query(consulta, engine)

    # Verificar si existen datos para esa hora
    if datos_existentes.empty:
        # Obtener los nuevos valores para la hora específica
        nuevos_datos = df[df['Timestamp'] == hora_especifica].copy()

        # Verificar si hay valores NaN en alguna columna
        if nuevos_datos.isnull().values.any():
            # Obtener los datos de la hora anterior
            hora_anterior = hora_especifica - pd.Timedelta(minutes=5)
            consulta_anterior = f"SELECT * FROM Data_sensores WHERE Timestamp = '{hora_anterior}'"
            datos_anteriores = pd.read_sql_query(consulta_anterior, engine)

            # Verificar si existen datos para la hora anterior
            if not datos_anteriores.empty:
                # Obtener la fila de la hora anterior
                fila_anterior = datos_anteriores.iloc[0]

                # Actualizar los valores NaN en todas las columnas
                columnas_actualizar = ['CO2', 'Temperatura', 'Humedad', 'Ocupacion', 'Icl', 'IDA',
                                       'Temperatura Radiante ITRC',
                                       'pmv', 'ppd', 'Categoria', 'Estrellas']
                for columna in columnas_actualizar:
                    nuevos_datos.loc[nuevos_datos[columna].isnull(), columna] = fila_anterior[columna]

        # Get the current max id in the table
        with engine.connect() as connection:
            result = connection.execute(text("SELECT MAX(`index`) FROM Data_sensores"))
            max_index = result.first()[0]  # This will be None if the table is empty

        # If the table is empty, start from 1, otherwise start from max_id + 1
        start_index = 1 if max_index is None else max_index + 1

        # Add the id column
        nuevos_datos['index'] = range(start_index, start_index + len(nuevos_datos))
        
        # Realizar la actualización en la base de datos con los nuevos valores
        nuevos_datos.to_sql('Data_sensores', engine, if_exists='append', index=False)
        print(f"Se subieron los datos para la hora {hora_especifica}.")
    else:
        print(f"Ya existen datos para la hora {hora_especifica}. No se subieron nuevos datos.")

Se subieron los datos para la hora 2023-07-11 14:15:00.
Se subieron los datos para la hora 2023-07-11 14:20:00.
Se subieron los datos para la hora 2023-07-11 14:25:00.
Se subieron los datos para la hora 2023-07-11 14:30:00.
